In [1]:
""" Ingest drought risk data in earthengine.
-------------------------------------------------------------------------------

files have been manually renamed to 
hazard
exposure
vulnaribility
risk
desertcoldareamask


Author: Rutger Hofste
Date: 201809028
Kernel: python27
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:
    



"""

SCRIPT_NAME = "Y2018M09D28_RH_DR_Ingest_EE_V01"
OUTPUT_VERSION = 3

OUTPUT_FILE_NAME = "df_errors.csv"
SEPARATOR = "_|-"

NODATA_VALUE =-9999

EXTRA_PROPERTIES = {"nodata_value":NODATA_VALUE ,
                    "ingested_by" : "RutgerHofste",
                    "script_used": SCRIPT_NAME,
                    "output_version":OUTPUT_VERSION,
                    "doi":"https://doi.org/10.1016/j.gloenvcha.2016.04.012"}


S3_INPUT_PATH = "s3://wri-projects/Aqueduct30/rawData/JRC/DroughtRisk/data/raw/"
GCS_OUTPUT_PATH = "gs://aqueduct30_v01/{}/".format(SCRIPT_NAME)


ee_output_path = "projects/WRI-Aquaduct/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_input_path = "/volumes/data/{}/input_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)

print("S3_INPUT_PATH: " + S3_INPUT_PATH +
      "\nec2_output_path" + ec2_output_path +
      "\nGCS_OUTPUT_PATH: " + GCS_OUTPUT_PATH +
      "\nee_output_path: " + ee_output_path )


S3_INPUT_PATH: s3://wri-projects/Aqueduct30/rawData/JRC/DroughtRisk/data/raw/
ec2_output_path/volumes/data/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03
GCS_OUTPUT_PATH: gs://aqueduct30_v01/Y2018M09D28_RH_DR_Ingest_EE_V01/
ee_output_path: projects/WRI-Aquaduct/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M09D28 UTC 11:10


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
#imports
import subprocess
import datetime
import os
import time
import re
import pandas as pd
import numpy as np
from datetime import timedelta
from osgeo import gdal
import aqueduct3

In [4]:
!rm -r {ec2_input_path}
!rm -r {ec2_output_path}
!mkdir -p {ec2_input_path}
!mkdir -p {ec2_output_path}

In [5]:
!aws s3 cp {S3_INPUT_PATH} {ec2_input_path} --recursive

download: s3://wri-projects/Aqueduct30/rawData/JRC/DroughtRisk/data/raw/desertcoldareamask.tif to ../../../../data/Y2018M09D28_RH_DR_Ingest_EE_V01/input_V03/desertcoldareamask.tif
download: s3://wri-projects/Aqueduct30/rawData/JRC/DroughtRisk/data/raw/risk.tif to ../../../../data/Y2018M09D28_RH_DR_Ingest_EE_V01/input_V03/risk.tif
download: s3://wri-projects/Aqueduct30/rawData/JRC/DroughtRisk/data/raw/exposure.tif to ../../../../data/Y2018M09D28_RH_DR_Ingest_EE_V01/input_V03/exposure.tif
download: s3://wri-projects/Aqueduct30/rawData/JRC/DroughtRisk/data/raw/hazard.tif to ../../../../data/Y2018M09D28_RH_DR_Ingest_EE_V01/input_V03/hazard.tif
download: s3://wri-projects/Aqueduct30/rawData/JRC/DroughtRisk/data/raw/vulnerability.tif to ../../../../data/Y2018M09D28_RH_DR_Ingest_EE_V01/input_V03/vulnerability.tif


In [6]:
files = os.listdir(ec2_input_path)

In [7]:
# Convert geotiff to all Float64 and with -9999 as NoData value
for one_file in files:
    input_file_path = os.path.join(ec2_input_path,one_file)
    output_file_path = os.path.join(ec2_output_path,one_file)
    xsize,ysize,geotransform,geoproj,Z = aqueduct3.read_gdal_file(input_file_path)
    
    Z[Z<-9990]= NODATA_VALUE 
    datatype=gdal.GDT_Float64
    
    aqueduct3.write_geotiff(output_file_path,geotransform,geoproj,Z,NODATA_VALUE,datatype)
    

In [8]:
!gsutil cp -r {ec2_output_path} {GCS_OUTPUT_PATH}

Copying file:///volumes/data/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03/hazard.tif [Content-Type=image/tiff]...
Copying file:///volumes/data/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03/exposure.tif [Content-Type=image/tiff]...
Copying file:///volumes/data/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03/risk.tif [Content-Type=image/tiff]...
Copying file:///volumes/data/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03/desertcoldareamask.tif [Content-Type=image/tiff]...
/ [4 files][ 27.1 MiB/ 27.1 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m -o ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file:///volumes/data/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03/vulnerability.tif [Content-Type=image/tiff]...
- [5 files][ 32.7 MiB/ 32.7 MiB]                                                
Operati

In [9]:
aqueduct3.earthengine.create_ee_folder_recursive(ee_output_path)

earthengine create folder projects/WRI-Aquaduct/Y2018M09D28_RH_DR_Ingest_EE_V01 b'Asset projects/WRI-Aquaduct/Y2018M09D28_RH_DR_Ingest_EE_V01 already exists\n'
earthengine create folder projects/WRI-Aquaduct/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03 b'Asset projects/WRI-Aquaduct/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03 already exists\n'


b'Asset projects/WRI-Aquaduct/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03 already exists\n'

In [10]:
files = os.listdir(ec2_input_path)

In [11]:
for one_file in files:    
    filename, extension = one_file.split(".")
    geotiff_gcs_path = GCS_OUTPUT_PATH  + "output_V{:02.0f}".format(OUTPUT_VERSION) + "/" + one_file
    output_ee_asset_id = ee_output_path + "/" + filename
    metadata_command = aqueduct3.dictionary_to_EE_upload_command(EXTRA_PROPERTIES)
    command = "/opt/anaconda3/bin/earthengine upload image --asset_id {} {}".format(output_ee_asset_id,geotiff_gcs_path)
    command = command + metadata_command
    print(command)
    response = subprocess.check_output(command, shell=True)

/opt/anaconda3/bin/earthengine upload image --asset_id projects/WRI-Aquaduct/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03/hazard gs://aqueduct30_v01/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03/hazard.tif -p doi=https://doi.org/10.1016/j.gloenvcha.2016.04.012 -p ingested_by=RutgerHofste -p script_used=Y2018M09D28_RH_DR_Ingest_EE_V01 -p output_version=3 --nodata_value=-9999
/opt/anaconda3/bin/earthengine upload image --asset_id projects/WRI-Aquaduct/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03/exposure gs://aqueduct30_v01/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03/exposure.tif -p doi=https://doi.org/10.1016/j.gloenvcha.2016.04.012 -p ingested_by=RutgerHofste -p script_used=Y2018M09D28_RH_DR_Ingest_EE_V01 -p output_version=3 --nodata_value=-9999
/opt/anaconda3/bin/earthengine upload image --asset_id projects/WRI-Aquaduct/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03/risk gs://aqueduct30_v01/Y2018M09D28_RH_DR_Ingest_EE_V01/output_V03/risk.tif -p doi=https://doi.org/10.1016/j.gloenvcha.2016.04.012 

In [12]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:00:46.053674


Previous Runs:  
0:00:38.999067